In [ ]:
!apt install python3.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3.7 is already the newest version (3.7.13-1+bionic3).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
!pip install -U torchtext==0.6.0

In [ ]:
!python -m spacy download en
!python -m spacy download de
!python -m spacy download fr

     |████████████████████████████████| 12.0 MB 10.5 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9 MB 12.8 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')
     |████████████████████████████████| 14.7 MB 27.0 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/fr_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/fr
You can now load the model via

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [ ]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [ ]:
def tokenize_de(text):
  return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
  return [tok.text for tok in spacy_en.tokenizer(text)][::-1]

In [ ]:
SRC = Field(tokenize = tokenize_de,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

TRG = Field(tokenize = tokenize_en,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), fields = (SRC, TRG))

In [ ]:
print(f'Number of training examples : {len(train_data.examples)}')
print(f'Number of validation examples : {len(valid_data.examples)}')
print(f'Number of testing examples : {len(test_data.examples)}')

Number of training examples : 29000
Number of validation examples : 1014
Number of testing examples : 1000


In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [ ]:
print(f'Unique tokens in source (de) vocabulary : {len(SRC.vocab)}')
print(f'Unique tokens in target (en) vocabulary : {len(TRG.vocab)}')

Unique tokens in source (de) vocabulary : 7855
Unique tokens in target (en) vocabulary : 5893


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

### Encoder

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FFcbF5%2FbtqEZ02FqMj%2F4JnntHWaEpkPLZjpUWSR50%2Fimg.png)

- input sequence를 넣어 최종적으로 context vector가 output으로 나옴.
- token들을 embedding layer를 통과시켜 embedding space에 mapping함.
- embedding된 vector를 lstm을 통과시켜 최종 hidden(context) vector를 생성함.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout): 
                       # input, embedding(mapping into space of D), hidden size, LSTM, Dropout
        super(Encoder, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, embedding_dim) # input dim will be mapped into embedding dimension
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout) # embedding dim -> hidden dim, num_layer, dropout
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        # src_size = [sequence_length, batch size]
        
        embedded = self.dropout(self.embedding(src))

        
        #embedded = [sequence_length, batch size, emb dim] foward to embedding layer -> add dimension of tensor(embedding size)
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions] 

        #hidden = [n layers * n directions, batch size, hid dim]

        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

### Decoder

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FFcbF5%2FbtqEZ02FqMj%2F4JnntHWaEpkPLZjpUWSR50%2Fimg.png)

- Enocder에서 출력된 hidden(context) vector가 decoder로 전달.
- Decoder의 첫 번째 input은 sos token이 들어감.
- 전달받은 hidden(context) vector와 input으로 등장할 확률이 제일 높은 단어를 출력함.
- training시에는 teacher forcing 방식으로 훈련함.(정확한 data로 훈련하기 위해 실제 값을 매번 input으로 사용)
- test시에는 context vector와 sos token으로 첫 단어를 예측하고 그 단어를 다음 input으로 사용

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout): # input_dim = output_dim
        super(Decoder, self).__init__()
        
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hidden_dim, output_dim) # output_dim -> vocab size
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size], but we want (1, batch) -> we predict one word at one time

        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0) # add 1-dimension
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, embedding dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell)) # output is just ouput, hidden and cell will be used to predict next word
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        # shape of prediction = [1, batch_size, length of vocab] --> prediction = [batch size, output dim]
        
        
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

In [ ]:
model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weight(m):
  for name, param in m.named_parameters():
    nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weight)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
optimizer = optim.Adam(model.parameters())

TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):

  model.eval()

  epoch_loss = 0

  with torch.no_grad():

    for i, batch in enumerate(iterator):
      src = batch.src
      trg = batch.trg

      output = model(src, trg, 0)
      # no teacher forcing
      # trg = [trg_len, batch_size, output_dim]

      output_dim = output.shape[-1]

      output = output[1:].view(-1, output_dim)
      trg = trg[1:].view(-1)
      # trg = [(trg_len - 1) * batch_size]
      # output = [(trg_len -1) * batch_size]

      loss = criterion(output, trg)
      epoch_loss += loss.item()

  return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
  
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')


Epoch: 01 | Time: 0m 35s
	Train Loss: 4.978 | Train PPL: 145.196
	 Val. Loss: 4.861 |  Val. PPL: 129.140
Epoch: 02 | Time: 0m 36s
	Train Loss: 4.464 | Train PPL:  86.842
	 Val. Loss: 4.817 |  Val. PPL: 123.617
Epoch: 03 | Time: 0m 36s
	Train Loss: 4.209 | Train PPL:  67.304
	 Val. Loss: 4.640 |  Val. PPL: 103.497
Epoch: 04 | Time: 0m 36s
	Train Loss: 4.018 | Train PPL:  55.582
	 Val. Loss: 4.549 |  Val. PPL:  94.497
Epoch: 05 | Time: 0m 36s
	Train Loss: 3.843 | Train PPL:  46.683
	 Val. Loss: 4.406 |  Val. PPL:  81.957
Epoch: 06 | Time: 0m 36s
	Train Loss: 3.608 | Train PPL:  36.899
	 Val. Loss: 4.156 |  Val. PPL:  63.816
Epoch: 07 | Time: 0m 36s
	Train Loss: 3.427 | Train PPL:  30.779
	 Val. Loss: 4.003 |  Val. PPL:  54.787
Epoch: 08 | Time: 0m 36s
	Train Loss: 3.216 | Train PPL:  24.931
	 Val. Loss: 3.954 |  Val. PPL:  52.144
Epoch: 09 | Time: 0m 36s
	Train Loss: 3.060 | Train PPL:  21.335
	 Val. Loss: 3.858 |  Val. PPL:  47.365
Epoch: 10 | Time: 0m 36s
	Train Loss: 2.923 | Train PPL

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.807 | Test PPL:  44.994 |
